`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
 

# Лабораторная работа №5: Машины опорных векторов. 


В практических примерах ниже показано:    
* как классифицировать данные с помощью модели SVM
* как использовать конвейеры для подгонки модели и применения её к новым данным

Точность всех моделей оценивается методом перекрёстной проверки по 5 блокам.  


## Данные

1.   `fixed_acidity` – постоянная кислотность;
2.   `volatile_acidity` – переменная кислотность;
3.   `citric_acid` – содержание лимонной кислоты;
4.   `residual_sugar` - остаточный сахар;
5.   `chlorides` – содержание хлоридов;
6.   `free_sulfur_dioxide` – содержание диоксида серы в свободном виде;
7.   `total_sulfur_dioxide` – общее содержание диоксида серы;
8.   `density` – плотность;
9.   `pH` – кислотность;
10.  `sulphates` – содержание сульфатов;
11.  `alcohol` – содержание алкоголя;
12.  `quality` – балльная оценка качества вина, от 0 до 10;
13.  `Y` – целевая переменная: 1 = высокое качество (quality > 5), 0 = низкое (quality <= 5).

# Указания к выполнению


## Загружаем пакеты

In [28]:
# загрузка пакетов: инструменты --------------------------------------------
#  работа с массивами
import numpy as np
#  фреймы данных
import pandas as pd
#  графики
import matplotlib as mpl
#  стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# перекодировка символьных показателей
from sklearn.preprocessing import LabelEncoder
# загрузка файлов по URL
import urllib
# проверка существования файла на диске
from pathlib import Path
# для форматирования результатов с помощью Markdown
from IPython.display import Markdown, display
# перекодировка категориальных переменных
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# хи-квадрат тест на независимость по таблице сопряжённости
from scipy.stats import chi2_contingency, t
#  для таймера
import time

# загрузка пакетов: модели ------------------------------------------------- 
# SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression 
# стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split 
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report

# загрузка пакетов: данные -------------------------------------------------
from sklearn import datasets

# загрузка пакетов: модели -------------------------------------------------
#  дерево классификации
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree


# бэггинг
from sklearn.ensemble import BaggingClassifier
# случайный лес
from sklearn.ensemble import RandomForestClassifier
# бустинг
from sklearn.ensemble import GradientBoostingClassifier

# для теста хи-квадрат
import scipy

In [29]:
# константы
#  ядро для генератора случайных чисел
my_seed = 6
#  создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
#  примеры стилей и шаблонов графиков: 
#  http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

In [30]:
# загружаем таблицу и превращаем её во фрейм
DF_raw = pd.read_csv('https://raw.githubusercontent.com/ania607/ML/main/data/winequality-white_for_lab.csv')
DF_raw = DF_raw.drop(['quality'], axis=1)
# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)

Число строк и столбцов в наборе данных:
 (4898, 12)


In [31]:
DF_raw.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1


In [32]:
# типы столбцов
DF_raw.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
Y                         int64
dtype: object

In [33]:
DF_raw.isna().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
Y                       0
dtype: int64

In [34]:
DF = DF_raw.sample(frac = 0.85, random_state = my_seed)
DF_predict = DF_raw.drop(DF.index)

In [35]:
DF.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
count,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000
mean,6.849135,0.278086,0.333541,6.366731,0.045917,35.201417,138.328369,0.994012,3.189784,0.489926,10.522423,0.664425
std,0.844912,0.100482,0.118623,5.088573,0.022027,17.003168,42.613313,0.003013,0.151397,0.113545,1.233513,0.472248
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,0.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991700,3.090000,0.410000,9.500000,0.000000
50%,6.800000,0.260000,0.320000,5.100000,0.043000,34.000000,134.000000,0.993700,3.180000,0.480000,10.400000,1.000000
75%,7.300000,0.320000,0.380000,9.800000,0.050000,45.000000,168.000000,0.996100,3.280000,0.550000,11.400000,1.000000
max,14.200000,1.100000,1.230000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,1.000000


# Преобразование исходных данных и построение моделей
## Стандартизация и переход к главным компонентам

В качестве альтернативных моделей рассмотрим SVM с различными вариантами ядер и логистическую регрессию. Причём предварительно преобразуем пространство исходных показателей с помощью метода главных компонент.

In [36]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 0:12].values)

# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)

# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
    np.around(frac_var_expl, 3),
    '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.279 0.13  0.115 0.089 0.081 0.078 0.065 0.06  0.048 0.03  0.024 0.002] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 40.9% разброса 11 объясняющих переменных.
Теперь объединим функции-преобразователи и оценщики в конвейер с помощью `Pipeline` и оценим точность логистической регрессии с помощью перекрёстной проверки.

## Модель логистической регрессии с перекрёстной проверкой

In [37]:
# данные для обучения моделей
X_train = DF.iloc[:, 0:12] 
y_train = DF.iloc[:, -1]

# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 2),
                        LogisticRegression(random_state = my_seed, 
                                           solver = 'lbfgs'))

# будем сохранять точность моделей в один массив
score = list()
score_models = list()

# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator = pipe_lr, X = X_train, y = y_train, 
                     cv = 5, scoring='accuracy')

# записываем точность
score.append(np.around(np.mean(cv), 3)) 
score_models.append('sc_pca_logit')
print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели sc_pca_logit : 0.699


In [38]:
X_train

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,Y
3765,6.9,0.40,0.17,12.90,0.033,59.0,186.0,0.99754,3.08,0.49,9.4,0
154,8.2,0.23,0.40,7.50,0.049,12.0,76.0,0.99660,3.06,0.84,9.7,1
1383,7.1,0.36,0.56,1.30,0.046,25.0,102.0,0.99230,3.24,0.33,10.5,1
568,6.1,0.26,0.51,2.20,0.050,61.0,154.0,0.99290,3.08,0.60,9.8,1
1338,6.7,0.18,0.30,6.40,0.048,40.0,251.0,0.99560,3.29,0.52,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4189,6.4,0.21,0.21,5.10,0.097,21.0,105.0,0.99390,3.07,0.46,9.6,0
1466,6.9,0.19,0.49,6.60,0.036,49.0,172.0,0.99320,3.20,0.27,11.5,1
4436,6.7,0.28,0.28,4.50,0.051,14.0,92.0,0.99224,3.36,0.58,11.9,1
2125,6.0,0.28,0.22,12.15,0.048,42.0,163.0,0.99570,3.20,0.46,10.1,0


In [39]:
y_train

3765    0
154     1
1383    1
568     1
1338    0
       ..
4189    0
1466    1
4436    1
2125    0
3972    1
Name: Y, Length: 4163, dtype: int64



```
X_train и y_train определены правильно.
```



## SVM с перекрёстной проверкой

Построим несколько вариантов модели SVM с различными ядерными функциями.

In [40]:
pipe_svc = make_pipeline(StandardScaler(), 
                         SVC(random_state = my_seed))

# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1] 
param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__degree' : [2, 3],
               'svc__kernel': ['poly']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, 
                  scoring = 'accuracy', refit = True, cv = kfold, 
                  n_jobs = -1)

# таймер
tic = time.perf_counter()

# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)

# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 17.17 секунд


In [41]:
# точность лучшей модели
np.around(gs.best_score_, 3)

1.0

Точность > 96%, изменялся лист param_range.

In [42]:
# параметры лучшей модели
# * ядерная функция 
gs.best_estimator_.get_params()['svc__kernel']

'linear'

In [43]:
# * параметр регуляризации
gs.best_estimator_.get_params()['svc__C']

0.001

In [44]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid')
gs.best_estimator_.get_params()['svc__gamma']

'scale'

In [45]:
# * степень полинома (для ядра 'poly')
gs.best_estimator_.get_params()['svc__degree']

3

In [46]:
score.append(np.around(gs.best_score_,3))
score_models.append('sc_pca_svc')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[1],':',score[1])

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 1.0


# Метод kNN

Реализуем метод k-ближайших соседей с преобразованием PCA.

In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [48]:
k_range = list(range(1,32))
weight_options = ["uniform", "distance"]

pipe_knn = Pipeline(steps=[('standard', StandardScaler()), ('PCA', PCA(n_components = 5)), ('knn', KNeighborsClassifier())])
param_grid = {
    'knn__n_neighbors': list(range(1,32))
}

knn = KNeighborsClassifier()
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
grid = GridSearchCV(pipe_knn, param_grid, cv=kfold, scoring='accuracy')

tic = time.perf_counter()
grid.fit(X_train,y_train)
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 6.43 секунд


В ходе обучения модели я менял такие параметры как: n_splits и n_components в PCA (дало прибавку к точности с 0.699 до 0.954)

In [52]:
score.append(np.around(grid.best_score_,3))
score_models.append('sc_pca_knn')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[2],':',score[2])

Acc с перекрёстной проверкой 
для модели sc_pca_knn : 0.954



# Прогноз на отложенные наблюдения по лучшей модели

Ещё раз посмотрим на точность построенных моделей.  

In [50]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,sc_pca_logit,0.699
1,sc_pca_svc,1.000
2,sc_pca_knn,0.954


Все модели показывают хорошую точность по показателю $Acc$, при этом самой точной оказывается модель PCA + SVC. Сделаем прогноз на отложенные наблюдения.   

In [51]:
# прогноз с помощью лучшей модели ансамбля с SVC
y_hat=gs.best_estimator_.predict(X=DF_predict.iloc[:, 0:12])
# точность# характеристики точности
print(classification_report(DF_predict.iloc[:,-1],y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       243
           1       1.00      1.00      1.00       492

    accuracy                           1.00       735
   macro avg       1.00      1.00      1.00       735
weighted avg       1.00      1.00      1.00       735

